# Model Arima for Time Series Forecasting temperature

## LLibreries

In [1]:
import json
import os
import sys
import time
import datetime
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

In [2]:
# Directori de treball

# Ruta on esta el script actual
# __path__ = os.path.dirname(os.path.realpath(__file__)) # Si es fitxer .py
__path__ = os.getcwd() # Si es fitxer .ipynb

carpeta_dades = 'Dades_T_estacions_xema_Z1'
fitxer_dades = 'SCM_T_Z1.json'

path_dades = os.path.join(os.getcwd(), carpeta_dades, fitxer_dades)


    # Obrir un arxiu json per veure com esta estructurat i com organitzar les dades
with open(path_dades, 'r') as f:
    data = json.load(f)

    # Accedemos a las lecturas (temperaturas) del JSON
lecturas = data[0]['variables'][0]['lectures']

#     # Convertimos las lecturas a un DataFrame con solo las columnas necesarias
df = pd.DataFrame(lecturas)[['data', 'valor']]


# Convertimos la columna 'data' a tipo datetime para análisis posterior
df['data'] = pd.to_datetime(df['data'])

df.head()


,data,valor
0,1998-01-01 00:00:00+00:00,-4.6
1,1998-01-01 01:00:00+00:00,-5.1
2,1998-01-01 02:00:00+00:00,-5.3
3,1998-01-01 03:00:00+00:00,-5.5
4,1998-01-01 04:00:00+00:00,-5.8


In [3]:
# Agafar només les dades corresponents als hiverns i posar-les en un nou dataframe
# Les dates dels hiverns seran del 1 de desembre al 31 de març
df_hiverns = df[(df['data'].dt.month >= 11) | (df['data'].dt.month <= 2)]

df_hiverns.head()

,data,valor
0,1998-01-01 00:00:00+00:00,-4.6
1,1998-01-01 01:00:00+00:00,-5.1
2,1998-01-01 02:00:00+00:00,-5.3
3,1998-01-01 03:00:00+00:00,-5.5
4,1998-01-01 04:00:00+00:00,-5.8


In [9]:
# Estalblim les dades per entrenar el model fins al al hivern 2018-2019 inclos i testejarem amb les dades del hivern 2019-2020

# Per l'hivern 2018-2019 agafem fins nomes els mesos de gener i febrer del 2019. Els de novembre i desembre del 2019 ja formen part del següent hivern

df_train = df_hiverns[
    (df_hiverns['data'].dt.year < 2019) | 
    ((df_hiverns['data'].dt.year == 2019) & (df_hiverns['data'].dt.month <= 2))]


df_test = df_hiverns[
    ((df_hiverns['data'].dt.year == 2019) & (df_hiverns['data'].dt.month >= 11)) | 
    ((df_hiverns['data'].dt.year == 2020) & (df_hiverns['data'].dt.month <= 2))
]


In [ ]:
# ARIMA model

from statsforecast import StatsForecast
from statsforecast.models import AutoARIMA

# Creem el model AutoARIMA
model = AutoARIMA()